In [78]:
import os
import json
import sympy
import requests

import pyciemss
import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories

from mira.metamodel import *
from mira.modeling.amr.petrinet import AMRPetriNetModel, template_model_to_petrinet_json
from mira.sources.amr.petrinet import template_model_from_amr_json

In [79]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
MODEL_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/"

model1 = os.path.join(MODEL_PATH, "SEIRD_base_model01_petrinet.json")
model2 = os.path.join(MODEL_PATH, "SEIRHD_base_model01_petrinet.json")
model3 = os.path.join(MODEL_PATH, "LV_sheep_foxes.json")

In [81]:
start_time = 0.0
end_time = 5.0
logging_step_size = 1
num_samples = 5
n = 10

In [82]:
result1 = pyciemss.sample(model1, end_time, logging_step_size, num_samples, start_time=start_time)
display(result1['data'].head())
result1['data'][['timepoint_id', 'sample_id', 'S_state']].to_csv('sir.csv', index= False) # , 'I_state', 'R_state']]


,timepoint_id,sample_id,timepoint_unknown,persistent_beta_param,persistent_death_param,persistent_gamma_param,persistent_I0_param,S_state,I_state,E_state,R_state,D_state,infected_observable_state,dead_observable_state
0,0,0,0.0,0.670833,0.00755,0.190441,12.6903,19339988.0,12.690300,40.000000,0.000000,0.000000,12.690300,0.000000
1,1,0,1.0,0.670833,0.00755,0.190441,12.6903,19339984.0,19.614979,40.878460,3.070022,0.023355,19.614979,0.023355
2,2,0,2.0,0.670833,0.00755,0.190441,12.6903,19339964.0,25.997267,45.454643,7.381477,0.056154,25.997267,0.056154
3,3,0,3.0,0.670833,0.00755,0.190441,12.6903,19339944.0,32.656544,52.860279,12.915119,0.098250,32.656544,0.098250
4,4,0,4.0,0.670833,0.00755,0.190441,12.6903,19339920.0,40.147495,62.813911,19.778463,0.150462,40.147495,0.150462


### Plot histogram per state combination


In [83]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from itertools import combinations
import networkx as nx
from pyciemss.visuals import plots, vega
bin_outliers = False
log = False

# loading SIR data
sir_dataset = pd.read_csv("sir.csv")


def process_dataset(sir_dataset, bin_outliers, log):
    """
    This function bins the columns in the dataset that includes "_state" and plots histograms for each column.
    
    Inputs:
    sir_dataset (pd.DataFrame): The dataset with states, timepoint_id and sample_id
    bin_outliers (bool): A boolean value indicating whether to bin outliers separately in their own bin.
    log (bool): A boolean value indicating whether to use a log scale for the bins.
    
    Outputs:
    sir_dataset (pd.DataFrame): The processed dataset with added bin columns.
    bin_dict (dict): A dictionary storing the min and max values of each bin per column.
    """
    # Determine the number of bins using Sturges' rule and the nu
    num_bins =  math.ceil(math.log2(sir_dataset.shape[0]))

    sir_dataset_state = sir_dataset.loc[:, sir_dataset.columns.str.contains('_state')]

    # dictionary to store the min and max values of each bin per column
    bin_dict = {}

    sir_dataset_output = sir_dataset.copy()

    for col in sir_dataset_state.columns:
        max_val = sir_dataset_output[col].max()
        min_val = sir_dataset_output[col].min()

        # Calculate the 5th and 95th percentiles
        p5, p95 = sir_dataset_output[col].quantile([0.05, 0.95])

        # get histogram in the logspace
        if log:
            if bin_outliers:
                # put all outliers in their own bin at the end (trying to avoid the massive histogram bin for the S_state)
                p5_log = np.log10(p5)
                p95_log = np.log10(p95)

                bin_edges = np.logspace(np.max([0.0, p5_log]), p95_log, num=num_bins)
                bin_edges = np.concatenate(([min_val], bin_edges, [max_val]))
            else:
                min_log= np.log10(min_val)
                max_log = np.log10(max_val)
                bin_edges = np.logspace(np.max([0.0, min_log]), max_log, num=num_bins)
        else:
            if bin_outliers:
                bin_edges = np.linspace(p5, p95, num=num_bins)
                bin_edges = np.concatenate(([min_val], bin_edges, [max_val]))
            else: 
                bin_edges = np.linspace(min_val, max_val, num=num_bins)

        # saving bins edges in dictionary by state and bin number
        bin_dict[col] = [{i:(bin_edges[i], bin_edges[i+1])} for i in range(len(bin_edges)-1)]
        # add column with the bin number per histogram
        sir_dataset_output[col + '_bin'] = sir_dataset_output[col].apply(lambda x: np.digitize(x, bin_edges))

    # create a new column that combines the bin information for each state
    sir_dataset_output['combined_bin'] = sir_dataset_output.apply(lambda row: '_'.join([f"{col.replace('_state_bin','')}{int(row[col])}" for col in sir_dataset_output.columns if '_bin' in col]), axis=1)

    # create a mapping from original labels to shorter labels
    unique_bins = sir_dataset_output['combined_bin'].unique()
    label_mapping = {bin: f'bin_{i}' for i, bin in enumerate(unique_bins)}
    sir_dataset_output['short_bin'] = sir_dataset_output['combined_bin'].map(label_mapping)

    # save the mapping to a CSV file
    pd.DataFrame(list(label_mapping.items()), columns=['original_label', 'short_label']).to_csv('label_mapping.csv', index=False)

    return sir_dataset_output, bin_dict




In [84]:

def remove_consecutive_duplicates(lst):
    if not lst:  # if the list is empty, return it
        return lst
    new_lst = [lst[0]]  # add the first item of lst to new_lst
    for item in lst[1:]:  # iterate over the rest of lst
        if item != new_lst[-1]:  # if the current item is not the same as the last item in new_lst
            new_lst.append(item)  # add it to new_lst
    return new_lst

def plot_paths(binned_data, *, n=None, fig_width=10):
    bins_list = []
    for sample_id in binned_data['sample_id'].unique()[:n]:
        sample_df = binned_data[binned_data['sample_id'] == sample_id]
        bins = sample_df['combined_bin'].values
        bins_list.append(' '.join(bins.tolist()))

    return bins_list



In [85]:
sir_dataset_output, bin_dict= process_dataset(sir_dataset, bin_outliers = False, log = True)
bins_list =  plot_paths(sir_dataset_output)[:10]
len(bins_list)
print(bins_list)

['S4 S4 S3 S3 S2 S1', 'S4 S4 S4 S4 S4 S3', 'S5 S5 S4 S4 S4 S4', 'S4 S4 S4 S3 S3 S2', 'S4 S4 S4 S3 S3 S2']


In [86]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
import sententree
import sententree_vega
import networkx as nx
import vega
import vl_convert

%aimport sententree
%aimport sententree_vega
%aimport vega

In [88]:
logging_keywords = ["TRACE", "DEBUG", "INFO", "LOG"]
def contains_any(line, keywords):
    return any([w in line for w in keywords])

def clean_line(l):
    no_date = " ".join(l.split(" ")[1:]).strip()
    return no_date.replace("::", "-").replace(":", "-")


In [89]:
# Install Graphviz (run this in a terminal if 'brew' is not accessible from Jupyter)
# !brew install graphviz

# Update the PATH environment variable
import os
os.environ['PATH'] += os.pathsep + '/usr/local/bin'

# Verify PATH update and executable
# The following should print the path to the 'dot' executable
!which dot

# Confirm that the 'dot' command works
# The following should print the version of Graphviz installed
!dot -V

2906.76s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/usr/local/bin/dot


2912.51s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


dot - graphviz version 12.0.0 (20240704.0754)


In [94]:
# %%prun -D build_sentence.profile

cluster = [clean_line(l) for l in bins_list if contains_any(l, logging_keywords)][:200]

G = sententree.build_sententree(bins_list,
                                min_support = 2,
                                num_exemplars=10,
                                tag_with=sententree.tag_words_with_occurrence)



In [95]:
G.edges(data=True)

OutEdgeDataView([('S4_1 S4_2', 'S4_3', {'weight': 3}), ('S4_1 S4_2', 'S3_1 S3_2 S2_1', {'weight': 1}), ('S4_3', 'S4_4', {'weight': 2}), ('S4_3', 'S3_1 S3_2 S2_1', {'weight': 1}), ('S3_1 S3_2 S2_1', 'S1_1', {'weight': 1})])

In [96]:
G.nodes

NodeView(('S4_1 S4_2', 'S4_3', 'S3_1 S3_2 S2_1', 'S4_4', 'S1_1'))

In [97]:
schema = sententree_vega.vega_sententree(G, w=800, h=300)
vega.save_schema(schema, "schema.vg.json")
with open("../figures/bin_occurance.png", "wb") as f:
    png = vl_convert.vega_to_png(schema, scale=4)
    f.write(png)

vega.display(schema, format="interactive")